In [ ]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.3.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.2 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scispacy 0.5.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.3.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.2.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.0
    Uninstalling spacy-3.3.0:
      Successfully uninstalled spacy-3.3.0


In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import spacy
import scispacy
import numpy as np

In [2]:
from spacy.tokens.doc import Doc
from spacy.tokens.token import Token
from spacy.tokens.span import Span
from pandas.core.frame import DataFrame
from numpy import ndarray

In [ ]:
gena_nlp = spacy.load("en_gena_sm")
gena_nlp.pipeline

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f4503ab8c90>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7f43c6fe31a0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f439eab59b0>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7f439eaf83c0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f439e5eb9d0>),
 ('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x7f439ea83910>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f439e5ebcd0>)]

# Normalize the entities in result file

In [ ]:
results_df = pd.read_csv("gena_data_raw_3.csv")
results_df.head(5)

,PMID,Sentence,E1,Type_E1,Relation,E2,Type_E2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,subcortical infarcts,DISEASE
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,cerebral autosomal recessive,DISEASE
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,hereditary SVD,DISEASE
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,recessive arteriopathy,DISEASE
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,leukoencephalopathy,DISEASE


In [ ]:
# read data
import pandas as pd
papers_df = pd.read_csv("final_papers_3.csv")
full_df = pd.merge(results_df, papers_df, how='left', left_on=['PMID'], right_on=['PMID'])
full_df = full_df[['PMID', 'Sentence', 'Abstract', 'E1', 'Type_E1', 'Relation', 'E2', 'Type_E2']]
full_df.head(50)

,PMID,Sentence,Abstract,E1,Type_E1,Relation,E2,Type_E2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Subcortical small vessel disease (SVD) is char...,Transforming growth factor beta,CHEBI,be dysregulated in,subcortical infarcts,DISEASE
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Subcortical small vessel disease (SVD) is char...,Transforming growth factor beta,CHEBI,be dysregulated in,cerebral autosomal recessive,DISEASE
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Subcortical small vessel disease (SVD) is char...,Transforming growth factor beta,CHEBI,be dysregulated in,hereditary SVD,DISEASE
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Subcortical small vessel disease (SVD) is char...,Transforming growth factor beta,CHEBI,be dysregulated in,recessive arteriopathy,DISEASE
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Subcortical small vessel disease (SVD) is char...,Transforming growth factor beta,CHEBI,be dysregulated in,leukoencephalopathy,DISEASE
5,28470822,The aim of this study was to characterize sign...,Subcortical small vessel disease (SVD) is char...,BMPs,CHEMICAL,have signaling abnormalities in,sporadic SVD,DISEASE
6,31792039,QUESTION 1: What nutritional/metabolic test(s)...,A 9-year-old boy presented with a 2-day histor...,preserved vitamin E levels?Vitamin E,CHEBI,be added to,rancherosSome vitamin E,CHEBI
7,31792039,QUESTION 1: What nutritional/metabolic test(s)...,A 9-year-old boy presented with a 2-day histor...,metabolic disorders,DISEASE,be associated with,respiratory chain)Maple syrup urine disease,CHEMICAL
8,31792039,QUESTION 1: What nutritional/metabolic test(s)...,A 9-year-old boy presented with a 2-day histor...,metabolic disorders,DISEASE,be associated with,respiratory chain)Maple syrup urine disease,DISEASE
9,31792039,QUESTION 1: What nutritional/metabolic test(s)...,A 9-year-old boy presented with a 2-day histor...,metabolic disorders,DISEASE,be associated with,MSUD,DISEASE


In [ ]:
def get_full_words(abbr:str, abstract:str, nlp)->str:
    """
    Get full words of a given abbreviation in sentence
    """
    # Find spans that contains abbreviation
    doc = nlp(abstract)
    abbr_phrase = None
    for sent in doc.sents:
      find_abbr = re.compile(f"(.+) (\(|\[){abbr}(\)|\])").findall(sent.text)
      if len(find_abbr) > 0:
        abbr_phrase = find_abbr[0][0]
    if not abbr_phrase:
      return abbr

    abbr_phrase = nlp(abbr_phrase)
    full_words = []
    i = len(abbr_phrase) - 1
    while i >= 0:
      if abbr_phrase[i].pos_ not in ['ADJ', 'NOUN', 'PROPN', 'PART', 'NUM']:
        break
      full_words.append(abbr_phrase[i].text)
      i -= 1

    return ' '.join(full_words[::-1]).strip()

In [ ]:
abbr_dict = {}
full_df = full_df.reset_index(drop=True)
# changes all abbreviations in dataframe to their full words
## Use get_full_words
print("Using get_full_words function:")
full_df['Full_E1'] = full_df['E1'].tolist()
full_df['Full_E2'] = full_df['E2'].tolist()

for idx, row in full_df.iterrows():
    abstract = row['Abstract']

    if idx%100 == 0:
      print(f"Row {idx}")
    
    # Full words of E1
    e1 = re.sub(r"total |further |other |many |much |several |some ", "", str(row['E1']))
    try:
      full_e1 = get_full_words(abbr=e1, abstract=abstract, nlp=gena_nlp)
      if e1 != full_e1:
        abbr_dict[e1] = full_e1
      full_df.loc[idx, "E1"] = e1
      full_df.loc[idx, "Full_E1"] = full_e1
    except:
      print(e1)


    # Full words of E2
    e2 = re.sub(r"total |further |other |many |much |several |some ", "", str(row['E2']))
    try:
      full_e2 = get_full_words(abbr=e2, abstract=abstract, nlp=gena_nlp)
      if e2 != full_e2:
        abbr_dict[e2] = full_e2
      full_df.loc[idx, "E2"] = e2
      full_df.loc[idx, "Full_E2"] = full_e2
    except:
      print(e2)

## Use dictionary
# print("Using abbreviation dictionary:")
# for idx, row in full_df.iterrows():
#     e1 = row['E1']
#     e2 = row['E2']

#     if idx%100 == 0:
#         print(f"Row {idx}")
    
#     full_df.loc[idx, 'E1'] = abbr_dict.get(e1, e1)
#     full_df.loc[idx, 'E2'] = abbr_dict.get(e2, e2)

# # Save to file csv
# full_df = full_df.drop_duplicates(keep='first')
# full_df.to_csv("nah_normalize.csv", index=False)

# Numbers of relation
# print(f"There are {len(full_df)} relations now.")
abbr_dict.items()

Using get_full_words function:
Row 0
respiratory chain)Maple syrup urine disease
respiratory chain)Maple syrup urine disease
Row 100
Row 200
Row 300
Row 400
receptor A(HTR1 A
5-HT(1B/1D/1F receptor agonist
Row 500
FU(OX
Row 600
Row 700
g. (+)-alpha-pinene
Row 800
selective 5-HT(2A receptor antagonist eplivanserin
selective 5-HT(2A receptor antagonist eplivanserin
nonselective serotonin 5-HT(2A/2C receptor antagonists
nonselective serotonin 5-HT(2A/2C receptor antagonists
5-HT(2A receptor
5-HT(2A receptor
Row 900
5-HT(2A receptor antagonist
5-HT(2A receptor antagonist
Row 1000
specific L-(3H glutamate binding
L-(3H glutamate
Row 1100
Row 1200
Row 1300
(+)-9'-allyl-ABA
(+)-9'-allyl-ABA
Polyglutamine Binding Peptide-1(QBP1
Row 1400
CB(2 receptors
CB(2 receptor agonists
CB(2 receptor agonists
Row 1500
cm(3
Row 1600
Row 1700
Row 1800
Row 1900
Row 2000
Row 2100
Cl(-
F(-
Row 2200
Row 2300
[D-Arg]-SP
[D-Arg]-SP
Row 2400
Row 2500
(±)-N-methyl-γ-[4-(trifluoromethyl)phenoxy]benzenepropanamine hyd

dict_items([('BMPs', 'bone morphogenetic proteins'), ('MSUD', 'maple syrup urine disease'), ('TRD', 'Treatment-resistant depression'), ('EPA', 'eicosapentaenoic acid'), ('NICCD', 'neonatal intrahepatic cholestasis'), ('CTLN2', 'adult-onset type II citrullinemia'), ('ASD', 'Autism Spectrum Disorder'), ('LT3', 'liothyronine'), ('COVID-19', '2019 novel coronavirus disease'), ('phenolic compounds', 'Alkylresorcinol'), ('ADP', 'Accidental dural puncture'), ('TB', 'tuberculosis'), ('EHA', 'Egg Hatch Assay'), ('VRD', 'NMDA-induced ventral root depolarization'), ('MPEP', 'pyridine'), ('ADHD', 'Attention-deficit/hyperactivity disorder'), ('LC-PUFA', 'omega-6 long-chain PUFA'), ('SRS', 'less smoking-related symptomatology'), ('CP', 'aphasia'), ('TTP', 'μg/day transdermal testosterone patch'), ('HSDD', 'hypoactive sexual desire disorder'), ('PS', 'paradoxical sleep'), ('AA', 'more arachidonic acid'), ('MDD', 'major depressive disorder'), ('long-acting dihydropyridine', 'nisoldipine'), ('nano-Se',

In [ ]:
!pip install periodictable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 752 kB 5.1 MB/s 


In [ ]:
import periodictable
## Use dictionary and periodictable
print("Using abbreviation dictionary:")
for idx, row in full_df.iterrows():
    if idx%100 == 0:
        print(f"Row {idx}")    
    e1 = row['E1']
    e2 = row['E2']

    # Use dictionary
    full_df.loc[idx, 'Full_E1'] = abbr_dict.get(e1, e1)
    full_df.loc[idx, 'Full_E2'] = abbr_dict.get(e2, e2)

    # Use periodictable
    letters_1 = re.compile(r'([A-Za-z]+)').findall(e1)
    if len(letters_1) == 1:
      for el in periodictable.elements:
        if letters_1[0] == el.symbol:
            full_df.loc[idx, 'Full_E1'] = el.name
    
    letters_2 = re.compile(r'([A-Za-z]+)').findall(e2)
    if len(letters_2) == 1:
      for el in periodictable.elements:
        if letters_2[0] == el.symbol:
            full_df.loc[idx, 'Full_E2'] = el.name

Using abbreviation dictionary:
Row 0
Row 100
Row 200
Row 300
Row 400
Row 500
Row 600
Row 700
Row 800
Row 900
Row 1000
Row 1100
Row 1200
Row 1300
Row 1400
Row 1500
Row 1600
Row 1700
Row 1800
Row 1900
Row 2000
Row 2100
Row 2200
Row 2300
Row 2400
Row 2500
Row 2600
Row 2700
Row 2800
Row 2900
Row 3000
Row 3100
Row 3200
Row 3300
Row 3400
Row 3500
Row 3600
Row 3700
Row 3800
Row 3900
Row 4000
Row 4100
Row 4200
Row 4300
Row 4400
Row 4500
Row 4600
Row 4700
Row 4800
Row 4900
Row 5000
Row 5100
Row 5200
Row 5300
Row 5400
Row 5500
Row 5600
Row 5700
Row 5800
Row 5900
Row 6000
Row 6100
Row 6200
Row 6300
Row 6400
Row 6500
Row 6600
Row 6700
Row 6800
Row 6900
Row 7000
Row 7100
Row 7200
Row 7300
Row 7400
Row 7500
Row 7600
Row 7700
Row 7800
Row 7900
Row 8000
Row 8100
Row 8200
Row 8300
Row 8400
Row 8500
Row 8600


In [ ]:
print(len(full_df))
full_df = full_df.drop_duplicates(subset=['Full_E1', 'Relation', 'Full_E2']).reset_index(drop=True)
print(len(full_df))

8692
7896


In [ ]:
full_df.to_csv("gena_data_normalized_3.csv", index=False)

# Get ID for all Entities from MeSH

In [ ]:
# Get new relations extracted
import pandas as pd
gena_normal = pd.read_csv("gena_data_normalized_3.csv")
gena_normal_relation = gena_normal[["PMID", "Sentence", "E1", "Type_E1", "Relation", "E2", "Type_E2", "Full_E1", "Full_E2"]]
gena_normal_relation["ID_1"] = [""]*len(gena_normal_relation)
gena_normal_relation["ID_2"] = [""]*len(gena_normal_relation)
gena_normal_relation["Synonyms_1"] = [""]*len(gena_normal_relation)
gena_normal_relation["Synonyms_2"] = [""]*len(gena_normal_relation)
gena_normal_relation = gena_normal_relation[["PMID", "Sentence", "ID_1", "E1", "Type_E1", "Full_E1", "Synonyms_1", "Relation", "ID_2", "E2", "Type_E2", "Full_E2", "Synonyms_2"]]

gena_normal_relation.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,Synonyms_2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,subcortical infarcts,DISEASE,subcortical infarcts,
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,cerebral autosomal recessive,DISEASE,cerebral autosomal recessive,
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,hereditary SVD,DISEASE,hereditary SVD,
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,recessive arteriopathy,DISEASE,recessive arteriopathy,
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,leukoencephalopathy,DISEASE,leukoencephalopathy,


In [ ]:
def find_mesh_info(term:str)->tuple:
    """
    Get MeSH info (ID, Title and Synonyms) of a given term
    """
    root_URL = "https://www.ncbi.nlm.nih.gov"
    URL = f"{root_URL}/mesh/?term=\"{term}\"+%5BText+Words%5D"
    results = BeautifulSoup(requests.get(URL).content, "html.parser")
    first_result = results.find("div", class_="rslt")
    if first_result:
        detailed_URL = root_URL + first_result.find("a")['href']
        results = BeautifulSoup(requests.get(detailed_URL).content, "html.parser")
    elif results.find("span", class_="icon", string="No items found."):
        return ("No_ID_MeSH", term, "No synonyms")

    title = results.find("h1", class_="title").text
    mesh_id = results.find("p", text=re.compile(r'MeSH Unique ID.*')).text.replace("MeSH Unique ID: ", "")
    results_ab = results.find("div", class_="rprt abstract")
    mesh_synonyms = results_ab.findChildren("ul", recursive=False)
    if len(mesh_synonyms) > 0:
        all_synonyms = [syn.text for syn in BeautifulSoup(str(mesh_synonyms[0]), "html.parser").find_all("li")]
        return (mesh_id, title, '|'.join(all_synonyms))
    else:
        return (mesh_id, title, "No synonyms")

In [ ]:
term ="meropenem"
term_lemma = ' '.join([token.lemma_ for token in gena_nlp(term)])
find_mesh_info(term_lemma)

('D000077731',
 'Meropenem',
 '3-(5-Dimethylcarbamoylpyrrolidin-3-ylthio)-6-(1-hydroxyethyl)-4-methyl-7-oxo-1-azabicyclo(3.2.0)hept-2-ene-2-carboxylic acid|Merrem|Ronem|Penem|SM 7338|SM-7338|SM7338')

In [ ]:
gena_normal_relation.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,Synonyms_2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,subcortical infarcts,DISEASE,subcortical infarcts,
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,cerebral autosomal recessive,DISEASE,cerebral autosomal recessive,
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,hereditary SVD,DISEASE,hereditary SVD,
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,recessive arteriopathy,DISEASE,recessive arteriopathy,
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,,Transforming growth factor beta,CHEBI,Transforming growth factor beta,,be dysregulated in,,leukoencephalopathy,DISEASE,leukoencephalopathy,


In [ ]:
from time import sleep
gena_normal_relation['MeSH_E1'] = ['']*len(gena_normal_relation)
gena_normal_relation['MeSH_E2'] = ['']*len(gena_normal_relation)

mesh_dict = {}

for idx, row in gena_normal_relation.iterrows():
    print(idx)
    if idx < 7759:
      continue
    if not mesh_dict.get(str(row['Full_E1']), False):
      try:
        mesh_results_1 = find_mesh_info(str(row['Full_E1']))
        sleep(2)
      except:
        sleep(120)
        mesh_results_1 = find_mesh_info(str(row['Full_E1']))
        sleep(2)
      mesh_dict[str(row['Full_E1'])] = mesh_results_1

    if not mesh_dict.get(str(row['Full_E2']), False):
      try:
        mesh_results_2 = find_mesh_info(str(row['Full_E2']))
        sleep(2)
      except:
        sleep(120)
        mesh_results_2 = find_mesh_info(str(row['Full_E2']))
        sleep(2)
      mesh_dict[str(row['Full_E2'])] = mesh_results_2
    
    # Fill into the dataframe
    # Entities 1
    gena_normal_relation.loc[idx, 'ID_1'] = mesh_dict[str(row['Full_E1'])][0]
    gena_normal_relation.loc[idx, 'MeSH_E1'] = mesh_dict[str(row['Full_E1'])][1]
    gena_normal_relation.loc[idx, 'Synonyms_1'] = mesh_dict[str(row['Full_E1'])][2]
    
    # Entities 2
    gena_normal_relation.loc[idx, 'ID_2'] = mesh_dict[str(row['Full_E2'])][0]
    gena_normal_relation.loc[idx, 'MeSH_E2'] = mesh_dict[str(row['Full_E2'])][1]
    gena_normal_relation.loc[idx, 'Synonyms_2'] = mesh_dict[str(row['Full_E2'])][2]


Streaming output truncated to the last 5000 lines.
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085

In [ ]:
gena_normal_relation.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,Synonyms_2,MeSH_E1,MeSH_E2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,D046589,subcortical infarcts,DISEASE,subcortical infarcts,"Dementia, Hereditary Multi-Infarct Type|Dement...",,
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,C563990,cerebral autosomal recessive,DISEASE,cerebral autosomal recessive,Alopecia|Cerebral Infarction|Spinal Diseases|L...,,
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,No_ID_MeSH,hereditary SVD,DISEASE,hereditary SVD,No synonyms,,
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,C563990,recessive arteriopathy,DISEASE,recessive arteriopathy,Alopecia|Cerebral Infarction|Spinal Diseases|L...,,
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,D054038,leukoencephalopathy,DISEASE,leukoencephalopathy,"Leukoencephalopathy Syndromes, Posterior|Syndr...",,


In [ ]:
print(len(gena_normal_relation[gena_normal_relation['ID_1'] == "No_ID_MeSH"]), len(gena_normal_relation[gena_normal_relation['ID_2'] == "No_ID_MeSH"]))
gena_normal_relation[["PMID", "Sentence", "ID_1", "E1", "Type_E1", "Full_E1", "MeSH_E1", "Synonyms_1", "Relation", "ID_2", "E2", "Type_E2", "Full_E2", "MeSH_E2", "Synonyms_2"]].to_csv("/content/drive/MyDrive/gena_with_ID_MESH_3.csv", index=False)

# import time
# time.sleep(5)

# from google.colab import files
# files.download(f'gena_with_ID_MESH.csv')

3781 3931


# Get ID from Ontologies

In [ ]:
relation_df = pd.read_csv("/content/drive/MyDrive/gena_with_ID_MESH_3.csv")
relation_df.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,NaN,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,D046589,subcortical infarcts,DISEASE,subcortical infarcts,NaN,"Dementia, Hereditary Multi-Infarct Type|Dement..."
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,NaN,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,C563990,cerebral autosomal recessive,DISEASE,cerebral autosomal recessive,NaN,Alopecia|Cerebral Infarction|Spinal Diseases|L...
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,NaN,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,No_ID_MeSH,hereditary SVD,DISEASE,hereditary SVD,NaN,No synonyms
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,NaN,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,C563990,recessive arteriopathy,DISEASE,recessive arteriopathy,NaN,Alopecia|Cerebral Infarction|Spinal Diseases|L...
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,D000077294,Transforming growth factor beta,CHEBI,Transforming growth factor beta,NaN,"Receptor, Transforming Growth Factor beta Type...",be dysregulated in,D054038,leukoencephalopathy,DISEASE,leukoencephalopathy,NaN,"Leukoencephalopathy Syndromes, Posterior|Syndr..."


In [ ]:
# FULL ONTOLOGY
def concat_df(file_name:str, all_ont_df:DataFrame)->DataFrame:
  """
  Concat new dataframe from a file to the given dataframe.
  """
  ont_df = pd.read_csv(file_name)[['Class ID', 'Preferred Label', 'Synonyms']]
  if file_name == "APADISORDERS.csv":
    ont_df['Preferred Label'] = ont_df['Preferred Label'].map(lambda x: x.replace("_", " ").lower())
  else:
    ont_df['Preferred Label'] = ont_df['Preferred Label'].map(lambda x: x.lower() if isinstance(x, str) else "")
  return pd.concat([all_ont_df, ont_df])

ontologies = ['SYMP.csv', 'DOID.csv', 'CHEBI.csv', 'APADISORDERS.csv', 'ASDPTO.csv', 'FMA.csv', 'MFOMD.csv']
full_ont_df = pd.read_csv(ontologies[0])[['Class ID', 'Preferred Label', 'Synonyms']]
full_ont_df['Preferred Label'] = full_ont_df['Preferred Label'].map(lambda x: x.lower())
for ont in ontologies[1:]:
  full_ont_df = concat_df(ont, full_ont_df)
full_ont_df.drop_duplicates(subset=['Class ID'], inplace=True, ignore_index=True)
full_ont_df["Vector"] = full_ont_df['Preferred Label'].map(lambda x: np.array(gena_nlp(x).vector))
full_ont_df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DtypeWarning: Columns (8,51,65,71,75,79) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DtypeWarning: Columns (30,31) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DtypeWarning: Columns (41,71,75,77,99,100,103,113,114,138,140,156,157,158,178,198) have mixed types.Specify dtype option on import or set low_memory=False.


,Class ID,Preferred Label,Synonyms,Vector
0,http://purl.obolibrary.org/obo/SYMP_0000149,sudden onset of severe chills,NaN,"[-0.07547201, 0.31105042, -0.02869026, -0.1448..."
1,http://purl.obolibrary.org/obo/SYMP_0000259,dry hacking cough,NaN,"[-0.08133, 0.46636868, -0.01648433, -0.0966516..."
2,http://purl.obolibrary.org/obo/SYMP_0000729,pulmonary consolidation,NaN,"[-0.022594575, 0.311132, 0.176519, -0.0930199,..."
3,http://purl.obolibrary.org/obo/SYMP_0000746,enanthem,NaN,"[0.397117, 0.552231, -0.245164, -0.408734, -0...."
4,http://purl.obolibrary.org/obo/SYMP_0000412,anxiety,NaN,"[-0.568638, -0.0604911, -0.0904757, -0.11993, ..."


In [ ]:
# Protein Dictionary
protein_df = pd.read_csv("PR.csv")
protein_dict = {}
for idx, row in protein_df.iterrows():
  if isinstance(row['Preferred Label'], str):
    synonyms = [syn.lower() for syn in (row["Synonyms"].split("|") if isinstance(row["Synonyms"], str) else [])]
    protein_dict[row['Class ID']] = [row['Preferred Label'].lower()] + synonyms
len(list(protein_dict.keys()))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,10,11,14,16,18,25,30,32,47,50,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


332991

In [ ]:
def find_protein(term:str, protein_dict:dict)->tuple:
  """
  Find protein name of a term if possible
  """
  for key in protein_dict.keys():
    for protein in protein_dict[key][1:]:
      if term in re.sub('[^A-Za-z0-9]+', ' ', protein).split():
        return (key, protein_dict[key][0], '|'.join(protein_dict[key][1:]) if len(protein_dict[key]) > 1 else "No Synonyms")
  return ("No_ID_MeSH", term, "No synonyms")

def cosine_similar(vec_1:ndarray, vec_2:ndarray):
  """
  Cosine distance of 2 vectors in SpaCy
  """
  return (vec_1.dot(vec_2))/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))

def most_similar(ont_df:DataFrame, term:str, protein_dict:dict, nlp)->tuple:
  """
  Get the most similar term of a given term 
  """
  term_vector = np.array(nlp(term).vector)
  max_result = ont_df["Vector"].map(lambda x: cosine_similar(term_vector, x))
  max_similar = max_result.max()
  max_index = max_result.argmax()

  if max_similar > 0.95:
    synonyms = ont_df.loc[max_index, 'Synonyms'] if isinstance(ont_df.loc[max_index, 'Synonyms'], str) else "No synonyms"
    return (ont_df.loc[max_index, 'Class ID'], ont_df.loc[max_index, 'Preferred Label'], synonyms)
  else:
    return find_protein(term, protein_dict)

In [ ]:
term = "n-6 polyunsaturated fatty acid"
most_similar(full_ont_df, term, protein_dict, gena_nlp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app


('http://purl.obolibrary.org/obo/CHEBI_26208',
 'polyunsaturated fatty acid',
 'polyunsaturated fatty acids|PUFA|PUFAs')

In [ ]:
ontology_dict = {}
for idx, row in relation_df.iterrows():
  print(idx)
  
  if (row["ID_1"] != "No_ID_MeSH" and row["ID_2"] != "No_ID_MeSH"):
    continue

  if row["ID_1"] == "No_ID_MeSH":
    if not ontology_dict.get(row['E1'], False):
      term_1 = ' '.join(word.lemma_ for word in gena_nlp(row["Full_E1"]))
      result_1 = most_similar(full_ont_df, term_1, protein_dict, gena_nlp)
      ontology_dict[row['E1']] = result_1
    relation_df.loc[idx, 'ID_1'], relation_df.loc[idx, 'MeSH_E1'], relation_df.loc[idx, 'Synonyms_1'] = ontology_dict[row['E1']]
  
  if row["ID_2"] == "No_ID_MeSH":
    if not ontology_dict.get(row['E2'], False):
      term_2 = ' '.join(word.lemma_ for word in gena_nlp(row["Full_E2"]))
      result_2 = most_similar(full_ont_df, term_2, protein_dict, gena_nlp)
      ontology_dict[row['E2']] = result_2
    relation_df.loc[idx, 'ID_2'], relation_df.loc[idx, 'MeSH_E2'], relation_df.loc[idx, 'Synonyms_2'] = ontology_dict[row['E2']]


0
1
2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app


3
4
5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app


6


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app


7892
7893
7894


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app


7895


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in float_scalars
  from ipykernel import kernelapp as app


In [ ]:
print(len(relation_df[relation_df['ID_1'] != "No_ID_MeSH"]), len(relation_df[relation_df['ID_2'] != "No_ID_MeSH"]))
relation_df.to_csv("/content/drive/MyDrive/gena_update_Ontologies_full_3.csv", index=False)

# import time
# time.sleep(5)
# from google.colab import files
# files.download('nah_update_Ontologies_full.csv')

4571 4432


# Create new ID for new terms

In [ ]:
relation_df = pd.concat([pd.read_csv("/content/drive/MyDrive/gena_update_Ontologies_full.csv"),
                         pd.read_csv("/content/drive/MyDrive/gena_update_Ontologies_full_2.csv"),
                         pd.read_csv("/content/drive/MyDrive/gena_update_Ontologies_full_3.csv")]).reset_index(drop=True)
relation_df.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
1,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
2,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
3,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
4,27584019,Obesity is associated with reduced testosteron...,D063766,Obesity,DISEASE,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",be associated with,No_ID_MeSH,reduced testosterone,CHEMICAL,reduced testosterone,reduced testosterone,No synonyms


In [ ]:
available_df = pd.read_csv("available_fad.csv")
available_df.columns = ['ID_1', 'E1', 'Relation', 'ID_2', 'E2']
available_df['Full_E1'], available_df['Full_E2'] = available_df['E1'].tolist(), available_df['E2'].tolist()
available_df['MeSH_E1'], available_df['MeSH_E2'] = available_df['E1'].tolist(), available_df['E2'].tolist()
available_df['Type_E1'] = ['CHEMICAL']*2434 + ['DISEASE']*(len(available_df)-2434)
available_df['Type_E2'] = ['DISEASE']*2434 + ['NUTRITION']*(len(available_df)-2434)

In [ ]:
available_df['PMID'] = ['BC5CDR']*2434 + ['DOID']*(len(available_df)-2434)
available_df['Sentence'] = available_df['Synonyms_1'] = available_df['Synonyms_2'] = ['']*len(available_df)
available_df = available_df[relation_df.columns]
relation_df = pd.concat([relation_df, available_df], axis=0).dropna().reset_index(drop=True)
relation_df.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
1,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
2,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
3,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
4,27584019,Obesity is associated with reduced testosteron...,D063766,Obesity,DISEASE,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",be associated with,No_ID_MeSH,reduced testosterone,CHEMICAL,reduced testosterone,reduced testosterone,No synonyms


In [ ]:
relation_df[relation_df['ID_1'].isna()]

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2


In [ ]:
new_count = 0
new_term_id_dict = {}
for idx, row in relation_df.iterrows():
  print(idx)
  if row['ID_1'].startswith("No"):
    if new_term_id_dict.get(row['E1']):
      relation_df.loc[idx, 'ID_1'] = new_term_id_dict[row['E1']]
    else:
      new_id = "NEW" + "{:05d}".format(new_count)
      relation_df.loc[idx, 'ID_1'] = new_id
      new_term_id_dict[row['E1']] = new_id
      new_count += 1

  if row['ID_2'].startswith("No"):
    if new_term_id_dict.get(row['E2']):
      relation_df.loc[idx, 'ID_2'] = new_term_id_dict[row['E2']]
    else:
      new_id = "NEW" + "{:05d}".format(new_count)
      relation_df.loc[idx, 'ID_2'] = new_id
      new_term_id_dict[row['E2']] = new_id
      new_count += 1
# Check if there is no-id terms
len(relation_df[relation_df['ID_1'] == 'No_ID_MeSH']), len(relation_df[relation_df['ID_2'] == 'No_ID_MeSH'])

Streaming output truncated to the last 5000 lines.
40553
40554
40555
40556
40557
40558
40559
40560
40561
40562
40563
40564
40565
40566
40567
40568
40569
40570
40571
40572
40573
40574
40575
40576
40577
40578
40579
40580
40581
40582
40583
40584
40585
40586
40587
40588
40589
40590
40591
40592
40593
40594
40595
40596
40597
40598
40599
40600
40601
40602
40603
40604
40605
40606
40607
40608
40609
40610
40611
40612
40613
40614
40615
40616
40617
40618
40619
40620
40621
40622
40623
40624
40625
40626
40627
40628
40629
40630
40631
40632
40633
40634
40635
40636
40637
40638
40639
40640
40641
40642
40643
40644
40645
40646
40647
40648
40649
40650
40651
40652
40653
40654
40655
40656
40657
40658
40659
40660
40661
40662
40663
40664
40665
40666
40667
40668
40669
40670
40671
40672
40673
40674
40675
40676
40677
40678
40679
40680
40681
40682
40683
40684
40685
40686
40687
40688
40689
40690
40691
40692
40693
40694
40695
40696
40697
40698
40699
40700
40701
40702
40703
40704
40705
40706
40707
40708
40709
40710
4

(0, 0)

In [ ]:
relation_df.to_csv("gena_with_ID_full.csv", index=False)

# Prepare data for Knowledge Graph

## ID and terms from papers corresponding

In [ ]:
term_df = relation_df[['ID_1', 'E1', 'Full_E1', 'MeSH_E1', 'Synonyms_1', 'Type_E1', 'ID_2', 'E2', 'Full_E2', 'MeSH_E2', 'Synonyms_2', 'Type_E2']].fillna({'Synonyms_1': 'No synonyms', 'Synonyms_2': 'No synonyms'})
term_df.columns = ['ID_1', 'Paper_term_1', 'Full_paper_term_1', 'Standard_term_1', 'Synonyms_1', 'Type_E1', 'ID_2', 'Paper_term_2', 'Full_paper_term_2', 'Standard_term_2', 'Synonyms_2', 'Type_E2']
term_df.head(5)

,ID_1,Paper_term_1,Full_paper_term_1,Standard_term_1,Synonyms_1,Type_E1,ID_2,Paper_term_2,Full_paper_term_2,Standard_term_2,Synonyms_2,Type_E2
0,C098107,Flibanserin,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,CHEMICAL,NEW00000,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms,MENTAL_HEALTH
1,C098107,Flibanserin,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,CHEMICAL,D020018,HSDD,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions...",DISEASE
2,NEW00001,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,CHEMICAL,NEW00000,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms,MENTAL_HEALTH
3,NEW00001,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,CHEMICAL,D020018,HSDD,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions...",DISEASE
4,D063766,Obesity,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",DISEASE,NEW00002,reduced testosterone,reduced testosterone,reduced testosterone,No synonyms,CHEMICAL


In [ ]:
unique_id = {}
for idx, row in term_df.iterrows():
    for k in range(1, 3):
        unique_id[row[f'ID_{k}']] = unique_id.get(row[f'ID_{k}'], {'preferred_terms': row[f'Standard_term_{k}'], 'label': row[f'Type_E{k}'], 'paper_terms': set(), 'synonyms': set()})
        if row[f'Paper_term_{k}'] != '':
          unique_id[row[f'ID_{k}']]['paper_terms'].add(row[f'Paper_term_{k}'])
        if row[f'Full_paper_term_{k}'] != '':
          unique_id[row[f'ID_{k}']]['paper_terms'].add(row[f'Full_paper_term_{k}'])
        if row[f'Synonyms_{k}'] != 'No synonyms' or row[f'Synonyms_{k}'] != '':
          unique_id[row[f'ID_{k}']]['synonyms'].update(row[f'Synonyms_{k}'].split('|'))

In [ ]:
terms_tuple = []
for id, terms in unique_id.items():
    terms_tuple.append((id, terms['label'], terms['preferred_terms'], '|'.join(list(terms['paper_terms'].union(terms['synonyms'])))))
ID_terms_df = pd.DataFrame(terms_tuple, columns=['ID', 'Label', 'Preferred label', 'Synonyms'])
ID_terms_df.head(5)

,ID,Label,Preferred label,Synonyms
0,C098107,CHEMICAL,flibanserin [Supplementary Concept],Flibanserin|Benzimidazoles|flibanserin
1,NEW00000,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...
2,D020018,DISEASE,"Sexual Dysfunctions, Psychological",|Sexual Aversion Disorder|Sexual Aversion Diso...
3,NEW00001,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist|No ...
4,D063766,DISEASE,Pediatric Obesity,"Overweight, Adolescent|Obesity, Infantile|Adol..."


In [ ]:
ID_terms_df.to_csv("ID_to_ENTITIES_final.csv", index=False)

## < ID_1, Relation, ID_2 > tuples

In [ ]:
id_relation_df = relation_df.dropna().drop_duplicates(subset=['ID_1', 'Relation', 'ID_2']).reset_index(drop=True)
id_relation_df.to_csv("gena_data_final.csv", index=False)

# Final Results

In [49]:
import pandas as pd

In [79]:
id_relation_df = pd.read_csv("gena_data_final.csv")
available_data = pd.read_csv("available_fad.csv")
id_terms_df = pd.read_csv("ID_to_ENTITIES_final.csv")

## 1. Numbers of unique relations

In [80]:
# All relations
print("Total relations:", len(id_relation_df))

Total relations: 43367


In [81]:
n_from_old_relations = len(available_data)

# New relations
print("Detected relations:", len(id_relation_df) - n_from_old_relations)

# Old relations
print("Available relations:", n_from_old_relations)

Detected relations: 40805
Available relations: 2562


## 2. Numbers of new relation from Old Entities

In [82]:
old_relations = id_relation_df.iloc[-n_from_old_relations:, :]
old_couple_dict = {(id_1, id_2): [relation] for (id_1, id_2, relation) in zip(old_relations['ID_1'], old_relations['ID_2'], old_relations['Relation'])}

new_relations = id_relation_df.iloc[:-n_from_old_relations, :]
for (id_1, id_2, relation) in zip(new_relations['ID_1'], new_relations['ID_2'], new_relations['Relation']):
  if (id_1, id_2) in old_couple_dict.keys():
    old_couple_dict[(id_1, id_2)].append(relation)

new_relation_from_old_tuples = []
for couple in old_couple_dict.keys():
  if len(old_couple_dict[couple]) > 1:
    for relation in old_couple_dict[couple][1:]:
      new_relation_from_old_tuples.append((couple[0], couple[1], old_couple_dict[couple][0], relation))

new_relation_from_old_df = pd.DataFrame(new_relation_from_old_tuples, columns=['ID_1', 'ID_2', 'Old_Relation', 'New_Relation'])
print(f"There are {len(new_relation_from_old_df)} relations for old couplez-entities.")
new_relation_from_old_df.head(len(new_relation_from_old_df))

There are 21 relations for old couplez-entities.


,ID_1,ID_2,Old_Relation,New_Relation
0,D003024,D000380,CID,cause fatal hematopoietic toxicity as
1,D017367,D001714,CID,be prescribed to patients with
2,D018021,D011141,CID,include
3,D003024,D009205,CID,be effective medication for
4,D004837,D006323,CID,have the use to
5,D010433,D012640,CID,induce
6,D007649,D006212,CID,cause
7,D007649,D006212,CID,be complicated by
8,D007649,D006212,CID,induce
9,D002110,D012640,CID,have the influence on


## 3. Numbers of totally new relations

In [83]:
totally_relations = len(new_relations) - len(new_relation_from_old_df)
print("Totally new relations:", totally_relations)

Totally new relations: 40784


## 4. Numbers of New Entities

In [85]:
print("Total numbers of entities:", len(id_terms_df))

Total numbers of entities: 28598


In [86]:
new_id = len(ID_terms_df[ID_terms_df['ID'].str.startswith('NEW')])
print("Numbers of new entities:", new_id)

Numbers of new entities: 21831


## 5. Precision

In [ ]:
result_1 = pd.read_excel("sample_test_final_R01.xlsx")
result_2 = pd.read_excel("sample_test_final_R02.xlsx")

### 5.a. Precision of Entities

In [ ]:
import sklearn
from sklearn.metrics import cohen_kappa_score

print("Precision of Entities by Annotator 1: ", (sum(result_1['E1_Result']) + sum(result_1['E2_Result']))/(2.0*len(result_1)))
print("Precision of Entities by Annotator 2: ", (sum(result_2['E1_Result']) + sum(result_2['E2_Result']))/(2.0*len(result_2)))

print("IAA of Entities: ", cohen_kappa_score(result_1['E1_Result'].tolist() + result_1['E2_Result'].tolist(), result_2['E1_Result'].tolist() + result_2['E2_Result'].tolist()))

Precision of Entities by Annotator 1:  0.952
Precision of Entities by Annotator 2:  0.958
IAA of Entities:  0.8836776474967429


### 5.b. Precision of Relations

In [ ]:
print("Precision of Relations by Annotator 1:", result_1['Relation_Result'].mean())
print("Precision of Relations by Annotator 2:", result_2['Relation_Result'].mean())

print("IAA of Entities: ", cohen_kappa_score(result_1['Relation_Result'].tolist(), result_2['Relation_Result']))

Precision of Relations by Annotator 1: 0.804
Precision of Relations by Annotator 2: 0.816
IAA of Entities:  0.8570834632574577
